In [1]:
import os
import pickle
import scipy.io
import numpy as np
from scfw.scopt import scopt
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
results_folder = './results/'

In [3]:
data_folder = '../data'

problem_name = 'syn_1000_1500_10_50'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [4]:
np.random.seed(42)

## Data preprocessing and parameters initialization

In [5]:
N, n = R.shape
Mf = 2
nu = 3

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-12

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
    'rho':np.sqrt(n), #parameters for ll00
    'diam_X':np.sqrt(2),
    'sigma_f':1,                   
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [6]:
func_x = lambda x: pr.portfolio(R,x)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
prox_func = lambda x, L: pr.proj_simplex(x)

In [7]:
points = [x0]
 
for _ in range(4):
    x = np.abs(np.random.rand(len(x0)))
    x = x / sum(x)
    points.append(x)

## Run FW

In [8]:
#run_alpha_policies = ["standard","line_search","icml"]
run_alpha_policies = ['new_lloo','lloo']


for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x, 
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
        'grad_hist': grad_hist
    }

********* Algorithm starts *********
5.232711198171899
iter = 1, stepsize = 0.3917318246754077, criterion = 0.01, upper_bound=0.05635009446772796, lower_bound=-5.176361103704171, real_Gap=5.232711198171899
iter = 1000, stepsize = 0.0011439559681520565, criterion = 0.0009248454983790988, upper_bound=-4.571409662600624, lower_bound=-4.571699455146176, real_Gap=0.0002897925455513928
iter = 2000, stepsize = 0.0007198143544010637, criterion = 0.0004535517143560335, upper_bound=-4.571412618402145, lower_bound=-4.571519133437572, real_Gap=0.00010651503542735696
iter = 3000, stepsize = 0.0004789259169793097, criterion = 0.0003023332497081548, upper_bound=-4.571413139630295, lower_bound=-4.571460408543107, real_Gap=4.7268912812370445e-05
iter = 4000, stepsize = 0.0005382770017464023, criterion = 0.0002257643857725455, upper_bound=-4.57141332361432, lower_bound=-4.571432137763861, real_Gap=1.8814149541412917e-05
iter = 5000, stepsize = 0.0002266682508409376, criterion = 0.00018066708752570985, u

iter = 46000, stepsize = 7.028002214314311e-05, criterion = 1.955196908310865e-05, upper_bound=-4.571413548739928, lower_bound=-4.571414674342945, real_Gap=1.1256030170159192e-06
iter = 47000, stepsize = 6.878406992311414e-05, criterion = 1.9132775007899974e-05, upper_bound=-4.571413548814043, lower_bound=-4.571414674342945, real_Gap=1.125528902079509e-06
iter = 48000, stepsize = 2.9812291999272484e-05, criterion = 1.8737074646568454e-05, upper_bound=-4.571413548868817, lower_bound=-4.571414674342945, real_Gap=1.125474128116366e-06
iter = 49000, stepsize = 4.3809824716439726e-05, criterion = 1.8350321968689234e-05, upper_bound=-4.571413548940388, lower_bound=-4.571414674342945, real_Gap=1.1254025569229498e-06
iter = 50000, stepsize = 4.2932687095796005e-05, criterion = 1.7985865285638962e-05, upper_bound=-4.571413548980502, lower_bound=-4.571414674342945, real_Gap=1.1253624432328024e-06
210.00003051757812
********* Algorithm starts *********
Convergence achieved!
iter = 1, stepsize = 6

In [9]:
run_alpha_policies = ["standard","line_search","icml"]

for policy in run_alpha_policies:
    policy_results = []
    for i in range(len(points)):
        x0 = points[i]
        x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                        grad_x,
                        grad_beta,
                        hess_mult_x,
                        extra_func,                                                    
                        Mf,
                        nu,
                        linear_oracle,                                                    
                        x0,
                        FW_params,                                       
                        alpha_policy=policy,                                                    
                        eps=terminate_tol,
                        print_every=1000, 
                        debug_info=False)
        
        policy_results.append(
            {
        'Q_hist': Q_hist,
        'time_hist': time_hist,}
        )
    with open('./results/portfolio/' + problem_name + '_' + policy + '.pckl', 'wb') as f:
        pickle.dump(policy_results, f)
#    results[problem_name][policy] = policy_results

********* Algorithm starts *********
iter = 1, stepsize = 0.6666666666666666, criterion = 0.01, upper_bound=0.05635009446772796, lower_bound=-5.176361103704171, real_Gap=5.232711198171899
iter = 1000, stepsize = 0.001996007984031936, criterion = 0.001419463780521935, upper_bound=-4.571404478778168, lower_bound=-4.571778436105625, real_Gap=0.0003739573274570418
iter = 2000, stepsize = 0.000999000999000999, criterion = 0.0007108677667445316, upper_bound=-4.571411308072072, lower_bound=-4.571608592299427, real_Gap=0.00019728422735454387
iter = 3000, stepsize = 0.0006662225183211193, criterion = 0.000473696169495033, upper_bound=-4.571412539815347, lower_bound=-4.571526779724484, real_Gap=0.00011423990913694837
iter = 4000, stepsize = 0.0004997501249375312, criterion = 0.0003552706199979203, upper_bound=-4.5714129990584675, lower_bound=-4.571447168541309, real_Gap=3.416948284140631e-05
iter = 5000, stepsize = 0.00039984006397441024, criterion = 0.00028416954354734583, upper_bound=-4.571413

iter = 46000, stepsize = 4.347637059258293e-05, criterion = 3.088584448692209e-05, upper_bound=-4.571413546227543, lower_bound=-4.571415789218469, real_Gap=2.2429909263266268e-06
iter = 47000, stepsize = 4.255138079230671e-05, criterion = 3.022818833563635e-05, upper_bound=-4.571413546366063, lower_bound=-4.571415789218469, real_Gap=2.2428524060202903e-06
iter = 48000, stepsize = 4.1664930627890504e-05, criterion = 2.9598839050545453e-05, upper_bound=-4.571413546590672, lower_bound=-4.571415789218469, real_Gap=2.2426277972442676e-06
iter = 49000, stepsize = 4.0814660626096896e-05, criterion = 2.8994533294479144e-05, upper_bound=-4.571413546700352, lower_bound=-4.571415789218469, real_Gap=2.2425181169793973e-06
iter = 50000, stepsize = 3.999840006399744e-05, criterion = 2.841472370678842e-05, upper_bound=-4.571413546941203, lower_bound=-4.571415789218469, real_Gap=2.2422772660846135e-06
172.1743562221527
********* Algorithm starts *********
iter = 1, stepsize = 0.6666666666666666, crite

iter = 41000, stepsize = 4.877810838495683e-05, criterion = 3.465223855539247e-05, upper_bound=-4.571413545055492, lower_bound=-4.571415789220932, real_Gap=2.244165440146162e-06
iter = 42000, stepsize = 4.7616780153326035e-05, criterion = 3.382695998900654e-05, upper_bound=-4.5714135454735425, lower_bound=-4.571415789220932, real_Gap=2.2437473896630422e-06
iter = 43000, stepsize = 4.650946467606158e-05, criterion = 3.304078591744731e-05, upper_bound=-4.571413545613828, lower_bound=-4.571415789220932, real_Gap=2.243607104546186e-06
iter = 44000, stepsize = 4.545247943275306e-05, criterion = 3.228948697216416e-05, upper_bound=-4.571413545879364, lower_bound=-4.571415789220932, real_Gap=2.2433415685085834e-06


## Run SCOPT

In [9]:
x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
        grad_x,
        hess_mult_x,
        hess_mult_vec_x,
        Mf,
        nu,
        prox_func,
        x0,  
        sc_params,                                              
        eps=terminate_tol,                                              
        print_every=1)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

In [10]:
with open(os.path.join(results_folder, 'portfolio', problem_name + '.pckl'), 'wb') as f:
    pickle.dump(results, f)